In [1]:
import cv2
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras import datasets, applications

In [2]:
#use this string for the path to the folder containing all the images in kaggle

folder_path = '/kaggle/input/ucf101/'

class_indices_file = open(folder_path+ 'UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/classInd.txt', 'r')
class_index = {}
for line in class_indices_file:
    class_ = line.split(' ')
    class_index[class_[1].strip('\n')] = int(class_[0])
    
class_indices_file.close()

print(class_index)

{'ApplyEyeMakeup': 1, 'ApplyLipstick': 2, 'Archery': 3, 'BabyCrawling': 4, 'BalanceBeam': 5, 'BandMarching': 6, 'BaseballPitch': 7, 'Basketball': 8, 'BasketballDunk': 9, 'BenchPress': 10, 'Biking': 11, 'Billiards': 12, 'BlowDryHair': 13, 'BlowingCandles': 14, 'BodyWeightSquats': 15, 'Bowling': 16, 'BoxingPunchingBag': 17, 'BoxingSpeedBag': 18, 'BreastStroke': 19, 'BrushingTeeth': 20, 'CleanAndJerk': 21, 'CliffDiving': 22, 'CricketBowling': 23, 'CricketShot': 24, 'CuttingInKitchen': 25, 'Diving': 26, 'Drumming': 27, 'Fencing': 28, 'FieldHockeyPenalty': 29, 'FloorGymnastics': 30, 'FrisbeeCatch': 31, 'FrontCrawl': 32, 'GolfSwing': 33, 'Haircut': 34, 'Hammering': 35, 'HammerThrow': 36, 'HandstandPushups': 37, 'HandstandWalking': 38, 'HeadMassage': 39, 'HighJump': 40, 'HorseRace': 41, 'HorseRiding': 42, 'HulaHoop': 43, 'IceDancing': 44, 'JavelinThrow': 45, 'JugglingBalls': 46, 'JumpingJack': 47, 'JumpRope': 48, 'Kayaking': 49, 'Knitting': 50, 'LongJump': 51, 'Lunges': 52, 'MilitaryParade': 

In [3]:
req_classes = [[1,2], [8,9], [23,24], [30], [87], [70], [72], [100], [59,60,61,62,63,64,65,66,67], 
               [83],[98], [29], [85], [79], [80], [84], [51], [45], [77], [73], [44]]

class_names = []
for _ in req_classes:
    classes = []
    for __ in _:
      classes.append([key for key in list(class_index.keys()) if class_index[key] == __][0])
    class_names.append(classes)
print(class_names)


[['ApplyEyeMakeup', 'ApplyLipstick'], ['Basketball', 'BasketballDunk'], ['CricketBowling', 'CricketShot'], ['FloorGymnastics'], ['SumoWrestling'], ['PullUps'], ['PushUps'], ['WritingOnBoard'], ['PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'PlayingViolin'], ['SkyDiving'], ['WalkingWithDog'], ['FieldHockeyPenalty'], ['SoccerPenalty'], ['Shotput'], ['SkateBoarding'], ['SoccerJuggling'], ['LongJump'], ['JavelinThrow'], ['SalsaSpin'], ['Rafting'], ['IceDancing']]


In [4]:
label_dict = {}

label = 1
for _ in req_classes:
    for id in _:
        label_dict[id] = label
    label += 1

print(label_dict)

{1: 1, 2: 1, 8: 2, 9: 2, 23: 3, 24: 3, 30: 4, 87: 5, 70: 6, 72: 7, 100: 8, 59: 9, 60: 9, 61: 9, 62: 9, 63: 9, 64: 9, 65: 9, 66: 9, 67: 9, 83: 10, 98: 11, 29: 12, 85: 13, 79: 14, 80: 15, 84: 16, 51: 17, 45: 18, 77: 19, 73: 20, 44: 21}


In [5]:
#considering only one test train split 01
train_path_file = open(folder_path + "UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/trainlist01.txt", "r")
train_path_list = []
train_y = []
# print(next(train_path_file))
list_req_class = [i for l in req_classes for i in l]
print(list_req_class)
for _ in train_path_file:
    path, class_num = _.split(' ')
    if (int(class_num.strip('\n')) in list_req_class):
        train_path_list.append(path)
        train_y.append(label_dict[int(class_num.strip('\n'))])
        #print(int(label_dict[class_num.strip('\n')]))
    
train_path_file.close()


[1, 2, 8, 9, 23, 24, 30, 87, 70, 72, 100, 59, 60, 61, 62, 63, 64, 65, 66, 67, 83, 98, 29, 85, 79, 80, 84, 51, 45, 77, 73, 44]


In [6]:
len(train_path_list)

3035

In [7]:
test_path_file = open(folder_path + "UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/testlist01.txt", "r")
test_path_list = []
test_y = []
# print(next(test_path_file))
for _ in test_path_file:
    path = _.strip('\n')
    a,b = path.split('/')
    if (class_index[a] in list_req_class):
        test_path_list.append(path)
        test_y.append(label_dict[class_index[a]])


test_path_file.close()



In [8]:
len(test_path_list)

1214

In [9]:
# Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
train_y_hot = pd.get_dummies(train_y)
test_y_hot = pd.get_dummies(test_y)

In [10]:
labels = np.unique(np.array(train_y))

sig_frame_dict = {}
for i in labels:
    with open('/kaggle/input/train-hist-frames/'+ 'class_'+ str(i) +'_sig_frames.txt', 'r') as readfile:
        contents = readfile.read()
        lines = contents.splitlines()
    for l in lines:
        #print(l)
        l_arr = l.split(" ", 1)
        sig_frame_dict['/kaggle/input/ucf101/UCF101/UCF-101/'+ l_arr[0]] = eval(l_arr[1])

    with open('/kaggle/input/test-hist-frames/'+ 'test_class_'+ str(i) +'_sig_frames.txt', 'r') as read_testfile:
        contents = read_testfile.read()
        lines = contents.splitlines()
    for l in lines:
        l_arr = l.split(" ", 1)
        sig_frame_dict['/kaggle/input/ucf101/UCF101/UCF-101/' + l_arr[0]] = eval(l_arr[1])
    


In [11]:
sig_frame_dict['/kaggle/input/ucf101/UCF101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi']

[0, 1, 4, 6, 7, 12, 16, 22, 28, 33, 37, 42, 44, 48, 53, 58, 64, 72, 76, 84]

# ## **data loader**

## for skip interval/random/adaptive frame selection

In [13]:

# cnn_base = keras.applications.resnet.ResNet50(weights="imagenet", include_top=False, input_shape= (224,224,3))

# cnn_out = keras.layers.GlobalMaxPool2D()(cnn_base)
# cnn = keras.Model(inputs=cnn_base.input, outputs=cnn_out)
# cnn.trainable = False

In [14]:
# cnn.summary()

In [12]:
def generate_frames_from_videos(video_path, frame_selection = 'skip'):

        #CAPTURING FRAMES after EVERY 20 FRAMES OF VIDEOS
#         print(video_path, type(video_path))
        video1 = cv2.VideoCapture(video_path)
        #total_frames = video1.get(cv2.CAP_PROP_FRAME_COUNT)
        
        if (frame_selection == 'skip'):
            frames_total = []
            frames_arr = []
            while video1.isOpened():
                ret, frame = video1.read()

                if not ret:  #reached end of video
                    break
                frames_total.append(frame)

            video1.release()
            total_frames = len(frames_total)
            #print(total_frames)
            interval = total_frames // 20

            for _ in range (total_frames):
                if _ % interval == 0:
                    resized_frame = cv2.resize(frames_total[_], (224, 224))
                    
                    resized_frame = tf.keras.applications.resnet.preprocess_input(resized_frame)
                    resized_frame = resized_frame.reshape(1,224,224,3)
                    frames_arr.append(cnn.predict(resized_frame, verbose = 0))
                if _ >= 19 * interval:
                    break

            frames_arr = np.squeeze(np.array(frames_arr), axis = 1)
            return frames_arr
        
        
        elif (frame_selection == 'random'):
            
            frames_total = []
            frames_arr = []
            while video1.isOpened():
                ret, frame = video1.read()

                if not ret:  #reached end of video
                    break
                frames_total.append(frame)

            video1.release()
            total_frames = len(frames_total)
            rand_list = random.sample(range(0,total_frames),20)

            for _ in sorted(rand_list):
                resized_frame = cv2.resize(frames_total[_], (224, 224))
                frames_arr.append(resized_frame)
            return frames_arr
        
        
        elif (frame_selection == 'hist_difference'):
            frames_num = sig_frame_dict[video_path]
            #print(len(frames_num))
            frame_count = 0
            frames_arr = []
            
            frames_total = []
            while video1.isOpened():
                ret, frame = video1.read()

                if not ret:  #reached end of video
                    break
                frames_total.append(frame)
                
            total_frames = len(frames_total)
            
            actual = []
            
            for i in range(len(frames_num)):
                ind = list(frames_num)[i]
                if ind < total_frames:
                    actual.append(ind)
                else:
                    int_ind = np.random.randint(0,total_frames)
                    actual.append(int_ind)
            if (len(actual)< 20):
                actual = actual + list(np.random.randint(0,total_frames,20-len(actual)))
                actual = sorted(actual)
            for ind in actual:
                resized_frame = cv2.resize(frames_total[int(ind)], (224, 224))
                frames_arr.append(resized_frame)
#                 resized_frame = tf.keras.applications.resnet.preprocess_input(resized_frame)
#                 resized_frame = resized_frame.reshape(1,224,224,3)
#                 frames_arr.append(cnn.predict(resized_frame, verbose = 0))

            video1.release()
#             frames_arr = np.squeeze(np.array(frames_arr), axis = 1)
            return frames_arr
            

In [13]:
# from sklearn.utils import shuffle
# train_path,train_labels=shuffle(train_path,train_labels, random_state=42)

In [14]:
z = generate_frames_from_videos('/kaggle/input/ucf101/UCF101/UCF-101/SalsaSpin/v_SalsaSpin_g14_c03.avi', 'hist_difference')
np.array(z).shape

(20, 224, 224, 3)

In [15]:
folder_path = '/kaggle/input/ucf101/'

#             print(str(folder_path + 'UCF101/UCF-101/' + str(video_path, 'UTF-8')))
frame_arr = generate_frames_from_videos(str(folder_path + 'UCF101/UCF-101/' + str(train_path_list[1])), 'hist_difference')

In [16]:
np.array(frame_arr).shape

(20, 224, 224, 3)

In [17]:
import os

In [18]:
def video_data_generator(X=train_path_list, Y=train_y_hot, frame_selection = 'hist_difference'):
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X_paths = [X[i] for i in indices]
        labels = [Y[i] for i in indices]

        folder_path = '/kaggle/input/ucf101/'
        for i in range (len(X)):
            video_path = X_paths[i]
            label= labels[i]
#             print(type(frame_selection))
            frame_arr = generate_frames_from_videos(str(os.path.join(folder_path, "UCF101/UCF-101/", str(video_path, 'UTF-8'))), frame_selection.decode())
            yield np.array(frame_arr), label



### making an instance of the generator for hist_difference

In [19]:
#genertor converts every argument into bytes

dataset_train = tf.data.Dataset.from_generator(
 video_data_generator,
 args = (train_path_list,train_y_hot,'hist_difference'),
 output_signature=(
     tf.TensorSpec(shape=(20, 224,224,3), dtype=tf.float64),
     tf.TensorSpec(shape=(21,), dtype=tf.int16)
     )
 ).batch(batch_size = 8).prefetch(tf.data.AUTOTUNE)
dataset_test = tf.data.Dataset.from_generator(
 video_data_generator,
 args = (test_path_list,test_y_hot,'hist_difference'),
 output_signature=(
     tf.TensorSpec(shape=(20, 224,224,3), dtype=tf.float64),
     tf.TensorSpec(shape=(21,))
     )
 ).batch(batch_size = 8).prefetch(tf.data.AUTOTUNE)

In [20]:
sig_frame_dict[folder_path+ 'UCF101/UCF-101/' + 'Basketball/v_Basketball_g09_c01.avi']

[0, 12, 24, 36, 37, 40, 43, 45, 47, 48, 56, 60, 65, 68, 72, 73, 75, 78, 80, 81]

## Functional API

## Attention

In [44]:
from tensorflow.keras.layers import Input, Dense, Permute, Multiply

In [16]:
eff_net_ptmodel = tf.keras.applications.efficientnet_v2.EfficientNetV2L(
    include_top=False,
    weights='imagenet'
)
eff_net_ptmodel.trainable = False

473176280/473176280 [==============================] - 3s 0us/step


In [18]:

# encoding the images -- the model
eff_out = keras.layers.GlobalAveragePooling2D()(eff_net_ptmodel.output)
encoder = keras.Model(inputs = eff_net_ptmodel.input, outputs= eff_out)


input_feature = keras.layers.Input(shape = [20, 224, 224, 3])


#attention layer connecting the 
encoded_img =keras.layers.TimeDistributed(encoder)(input_feature)

query_img = keras.layers.TimeDistributed(keras.layers.Dense(128, activation = 'relu'))(encoded_img)
key_img = keras.layers.TimeDistributed(keras.layers.Dense(128, activation = 'relu'))(encoded_img)

att_out = keras.layers.Attention()([query_img, key_img])

concat_layer = keras.layers.Concatenate()([att_out, encoded_img])

encoded_sequence = keras.layers.Bidirectional(keras.layers.LSTM(128))(concat_layer)

lstm_dense_td = keras.layers.Dense(64, activation = 'relu')(encoded_sequence)

outputs = keras.layers.Dense(len(req_classes), activation="softmax")(lstm_dense_td)

model_effnet_att_lstm = keras.Model(input_feature, outputs)

In [35]:
model_effnet_att_lstm.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 20, 224, 22  0           []                               
                                4, 3)]                                                            
                                                                                                  
 time_distributed_7 (TimeDistri  (None, 20, 1280)    117746848   ['input_4[0][0]']                
 buted)                                                                                           
                                                                                                  
 time_distributed_8 (TimeDistri  (None, 20, 128)     163968      ['time_distributed_7[0][0]']     
 buted)                                                                                     

In [36]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)

In [37]:
model_effnet_att_lstm.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam()
                     , metrics = ["accuracy"])

In [38]:
model_effnet_att_lstm.fit(dataset_train, epochs=5,validation_data=dataset_test)

Epoch 1/5


2023-04-21 20:10:49.314481: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_3/time_distributed_7/model_2/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


95/95 [==============================] - 653s 6s/step - loss: 0.7489 - accuracy: 0.8033 - val_loss: 0.4530 - val_accuracy: 0.8674
Epoch 2/5
95/95 [==============================] - 557s 6s/step - loss: 0.1092 - accuracy: 0.9700 - val_loss: 0.4556 - val_accuracy: 0.8575
Epoch 3/5
95/95 [==============================] - 586s 6s/step - loss: 0.0399 - accuracy: 0.9924 - val_loss: 0.4734 - val_accuracy: 0.8789
Epoch 4/5
95/95 [==============================] - 560s 6s/step - loss: 0.0234 - accuracy: 0.9947 - val_loss: 0.5378 - val_accuracy: 0.8624
Epoch 5/5
95/95 [==============================] - 553s 6s/step - loss: 0.0140 - accuracy: 0.9980 - val_loss: 0.5254 - val_accuracy: 0.8748


In [39]:
model_effnet_att_lstm.save("/kaggle/working/model_effnet_att_lstm1.h5")

## convlstm attention lstm

In [45]:
modelconvlstm = Sequential()

#channels_last --> format for the image (224,224,3)

#smaller kernel size => capture slower motions
#larger kernel size => capture faster motions


modelconvlstm = Sequential()
modelconvlstm.add(ConvLSTM2D(filters = 8, kernel_size = (5, 5), activation = 'tanh',data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True, input_shape = (20, 224, 224, 3)))

modelconvlstm.add(MaxPooling3D(pool_size=(1, 2, 4), padding='valid', data_format='channels_last'))
modelconvlstm.add(TimeDistributed(Dropout(0.2)))

modelconvlstm.add(ConvLSTM2D(filters = 16, kernel_size = (7, 7), activation = 'tanh', data_format = "channels_last",
                      recurrent_dropout=0.2, return_sequences=True))

modelconvlstm.add(MaxPooling3D(pool_size=(1, 2, 8), padding='valid', data_format='channels_last'))
modelconvlstm.add(TimeDistributed(Dropout(0.2)))

modelconvlstm.add(TimeDistributed(Flatten()))




In [46]:
x = Input(shape = [20, 224,224, 3])
encoded_img = modelconvlstm(x)


query_img =TimeDistributed(keras.layers.Dense(128, activation = 'relu'))(encoded_img)
key_img = TimeDistributed(keras.layers.Dense(128, activation = 'relu'))(encoded_img)

att_out = Attention()([query_img, key_img])

concat_layer = Concatenate()([att_out, encoded_img])

encoded_sequence = Bidirectional(keras.layers.LSTM(128))(concat_layer)

lstm_dense_td = Dense(64, activation = 'relu')(encoded_sequence)

outputs = Dense(len(req_classes), activation="softmax")(lstm_dense_td)

model_convlstm_att_lstm = keras.Model(x, outputs)

In [47]:
model_convlstm_att_lstm.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 224, 22  0           []                               
                                4, 3)]                                                            
                                                                                                  
 sequential_1 (Sequential)      (None, 20, 4992)     84160       ['input_1[0][0]']                
                                                                                                  
 time_distributed_3 (TimeDistri  (None, 20, 128)     639104      ['sequential_1[0][0]']           
 buted)                                                                                           
                                                                                              

In [48]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)

In [49]:
model_convlstm_att_lstm.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam()
                     , metrics = ["accuracy"])

In [50]:
hist_model_convlstm_att_lstm = model_convlstm_att_lstm.fit(dataset_train, epochs=5,validation_data=dataset_test,callbacks = [early_stopping_callback])

Epoch 1/5


2023-04-25 15:14:37.677067: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/sequential_1/conv_lstm2d/while/body/_1/model/sequential_1/conv_lstm2d/while/dropout_3/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


380/380 [==============================] - 592s 2s/step - loss: 2.5707 - accuracy: 0.3048 - val_loss: 2.3254 - val_accuracy: 0.3674
Epoch 2/5
380/380 [==============================] - 553s 1s/step - loss: 2.3648 - accuracy: 0.3367 - val_loss: 2.2973 - val_accuracy: 0.3534
Epoch 3/5
380/380 [==============================] - 549s 1s/step - loss: 2.1983 - accuracy: 0.3565 - val_loss: 2.2576 - val_accuracy: 0.3295
Epoch 4/5
380/380 [==============================] - 552s 1s/step - loss: 2.1451 - accuracy: 0.3710 - val_loss: 2.0971 - val_accuracy: 0.3913
Epoch 5/5
380/380 [==============================] - 535s 1s/step - loss: 1.9202 - accuracy: 0.4234 - val_loss: 2.0060 - val_accuracy: 0.3970


In [51]:
hist_model_convlstm_att_lstm = model_convlstm_att_lstm.fit(dataset_train, epochs=5,validation_data=dataset_test,callbacks = [early_stopping_callback])

Epoch 1/5
380/380 [==============================] - 559s 1s/step - loss: 1.8081 - accuracy: 0.4590 - val_loss: 2.0525 - val_accuracy: 0.3830
Epoch 2/5
380/380 [==============================] - 553s 1s/step - loss: 1.8212 - accuracy: 0.4445 - val_loss: 1.8859 - val_accuracy: 0.4325
Epoch 3/5
380/380 [==============================] - 537s 1s/step - loss: 1.5663 - accuracy: 0.5278 - val_loss: 1.9038 - val_accuracy: 0.4423
Epoch 4/5
380/380 [==============================] - 544s 1s/step - loss: 1.4827 - accuracy: 0.5437 - val_loss: 1.8767 - val_accuracy: 0.4399
Epoch 5/5
380/380 [==============================] - 547s 1s/step - loss: 1.3576 - accuracy: 0.5766 - val_loss: 1.8272 - val_accuracy: 0.4621


In [52]:
hist_model_convlstm_att_lstm = model_convlstm_att_lstm.fit(dataset_train, epochs=10,validation_data=dataset_test,callbacks = [early_stopping_callback])

Epoch 1/10
380/380 [==============================] - 548s 1s/step - loss: 1.4331 - accuracy: 0.5638 - val_loss: 1.8157 - val_accuracy: 0.4893
Epoch 2/10
380/380 [==============================] - 557s 1s/step - loss: 1.2000 - accuracy: 0.6234 - val_loss: 1.8634 - val_accuracy: 0.4852
Epoch 3/10
380/380 [==============================] - 556s 1s/step - loss: 1.0476 - accuracy: 0.6689 - val_loss: 1.8803 - val_accuracy: 0.4802
Epoch 4/10
380/380 [==============================] - 544s 1s/step - loss: 1.0527 - accuracy: 0.6735 - val_loss: 2.1230 - val_accuracy: 0.4325
Epoch 5/10
380/380 [==============================] - 521s 1s/step - loss: 0.8630 - accuracy: 0.7318 - val_loss: 1.9897 - val_accuracy: 0.4720
Epoch 6/10
380/380 [==============================] - 535s 1s/step - loss: 0.8153 - accuracy: 0.7394 - val_loss: 2.7600 - val_accuracy: 0.3081


## VGG  16 hist bilstm

In [24]:
vg_pret_model = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet'
)
vg_pret_model.trainable = False
model_pret_x = Sequential()
model_pret_x.add(TimeDistributed(vg_pret_model, input_shape= (20,224,224,3)))

model_pret_x.add(TimeDistributed(GlobalMaxPool2D()))

model_pret_x.add(Bidirectional(LSTM(128)))

model_pret_x.add(Dense(len(req_classes), activation = 'softmax'))


58889256/58889256 [==============================] - 0s 0us/step


In [25]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)

In [26]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.SGD()
                     , metrics = ["accuracy"])

In [27]:
model_pret_x.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 20, 7, 7, 512)    14714688  
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 20, 512)          0         
 tributed)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 256)              656384    
 l)                                                              
                                                                 
 dense (Dense)               (None, 21)                5397      
                                                                 
Total params: 15,376,469
Trainable params: 661,781
Non-trainable params: 14,714,688
______________________________________

In [28]:
hist_pret_x = model_pret_x.fit(dataset_train, epochs=15,validation_data=dataset_test)

Epoch 1/15
380/380 [==============================] - 476s 1s/step - loss: 2.0430 - accuracy: 0.4244 - val_loss: 1.7424 - val_accuracy: 0.4959
Epoch 2/15
380/380 [==============================] - 437s 1s/step - loss: 1.3605 - accuracy: 0.6471 - val_loss: 1.3893 - val_accuracy: 0.6516
Epoch 3/15
380/380 [==============================] - 422s 1s/step - loss: 1.0011 - accuracy: 0.7881 - val_loss: 1.0957 - val_accuracy: 0.6928
Epoch 4/15
380/380 [==============================] - 413s 1s/step - loss: 0.6985 - accuracy: 0.8774 - val_loss: 0.9156 - val_accuracy: 0.7578
Epoch 5/15
380/380 [==============================] - 423s 1s/step - loss: 0.5472 - accuracy: 0.9137 - val_loss: 0.8744 - val_accuracy: 0.7718
Epoch 6/15
380/380 [==============================] - 443s 1s/step - loss: 0.4439 - accuracy: 0.9367 - val_loss: 0.7824 - val_accuracy: 0.7891
Epoch 7/15
380/380 [==============================] - 422s 1s/step - loss: 0.3292 - accuracy: 0.9638 - val_loss: 0.7340 - val_accuracy: 0.7924

In [21]:
eff_net_ptmodel = tf.keras.applications.efficientnet_v2.EfficientNetV2L(
    include_top=False,
    weights='imagenet'
)
eff_net_ptmodel.trainable = False

473176280/473176280 [==============================] - 3s 0us/step


In [34]:

# encoding the images -- the model
eff_out = keras.layers.GlobalAveragePooling2D()(eff_net_ptmodel.output)
encoder = keras.Model(inputs = eff_net_ptmodel.input, outputs= eff_out)


input_feature = keras.layers.Input(shape = [20, 224, 224, 3])


#attention layer connecting the 
encoded_img =keras.layers.TimeDistributed(encoder)(input_feature)

lstm_out = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences = True))(encoded_img)

concat_layer = keras.layers.Concatenate()([lstm_out, encoded_img])


query_img = keras.layers.TimeDistributed(keras.layers.Dense(128, activation = 'relu'))(concat_layer)
key_img = keras.layers.TimeDistributed(keras.layers.Dense(128, activation = 'relu'))(concat_layer)

att_out = keras.layers.Attention()([query_img, key_img])

flatten = keras.layers.Flatten()(att_out)

lstm_dense_td = keras.layers.Dense(64, activation = 'relu')(flatten)

outputs = keras.layers.Dense(len(req_classes), activation="softmax")(lstm_dense_td)

model_effnet_lstm_att = keras.Model(input_feature, outputs)

In [35]:
model_effnet_lstm_att.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 20, 224, 22  0           []                               
                                4, 3)]                                                            
                                                                                                  
 time_distributed_11 (TimeDistr  (None, 20, 1280)    117746848   ['input_7[0][0]']                
 ibuted)                                                                                          
                                                                                                  
 bidirectional_5 (Bidirectional  (None, 20, 256)     1442816     ['time_distributed_11[0][0]']    
 )                                                                                          

In [36]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)

In [37]:
model_effnet_lstm_att.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam()
                     , metrics = ["accuracy"])

In [38]:
hist=model_effnet_lstm_att.fit(dataset_train, epochs=10,validation_data=dataset_test)

Epoch 1/10


2023-04-27 16:15:05.381158: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_9/time_distributed_11/model_8/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


380/380 [==============================] - 706s 2s/step - loss: 0.6939 - accuracy: 0.7924 - val_loss: 0.9626 - val_accuracy: 0.7586
Epoch 2/10
380/380 [==============================] - 582s 2s/step - loss: 0.1809 - accuracy: 0.9420 - val_loss: 0.8353 - val_accuracy: 0.8196
Epoch 3/10
380/380 [==============================] - 582s 2s/step - loss: 0.0994 - accuracy: 0.9733 - val_loss: 1.1433 - val_accuracy: 0.8188
Epoch 4/10
380/380 [==============================] - 594s 2s/step - loss: 0.1261 - accuracy: 0.9684 - val_loss: 1.3529 - val_accuracy: 0.7768
Epoch 5/10
380/380 [==============================] - 585s 2s/step - loss: 0.0994 - accuracy: 0.9727 - val_loss: 0.9995 - val_accuracy: 0.8377
Epoch 6/10
380/380 [==============================] - 602s 2s/step - loss: 0.0605 - accuracy: 0.9829 - val_loss: 1.3843 - val_accuracy: 0.8122
Epoch 7/10
380/380 [==============================] - 564s 1s/step - loss: 0.0434 - accuracy: 0.9868 - val_loss: 1.1564 - val_accuracy: 0.8229
Epoch 8/10

In [40]:
hist=model_effnet_lstm_att.fit(dataset_train, epochs=10,validation_data=dataset_test)

Epoch 1/10
380/380 [==============================] - 588s 2s/step - loss: 0.0826 - accuracy: 0.9786 - val_loss: 1.6860 - val_accuracy: 0.8435
Epoch 2/10
380/380 [==============================] - 573s 2s/step - loss: 0.0517 - accuracy: 0.9888 - val_loss: 1.4925 - val_accuracy: 0.8583
Epoch 3/10
380/380 [==============================] - 610s 2s/step - loss: 0.0192 - accuracy: 0.9941 - val_loss: 1.7822 - val_accuracy: 0.8402
Epoch 4/10
380/380 [==============================] - 640s 2s/step - loss: 0.0724 - accuracy: 0.9871 - val_loss: 1.7911 - val_accuracy: 0.8484
Epoch 5/10
380/380 [==============================] - 590s 2s/step - loss: 0.0381 - accuracy: 0.9951 - val_loss: 1.4279 - val_accuracy: 0.8517
Epoch 6/10
380/380 [==============================] - 592s 2s/step - loss: 0.0302 - accuracy: 0.9908 - val_loss: 1.6689 - val_accuracy: 0.8418
Epoch 7/10
380/380 [==============================] - 594s 2s/step - loss: 0.0862 - accuracy: 0.9845 - val_loss: 1.4450 - val_accuracy: 0.8361

In [41]:
model_effnet_lstm_att.save("/kaggle/working/model_effnet_lstm_att.h5")